#Set-Up

In [ ]:
#Copy-and-paste the code below to use as "set-up" when your optimization model uses Pyomo.
#Uncomment the appropriate solver that you need.
#for reference, see https://colab.research.google.com/drive/1yGk8RB5NXrcx9f1Tb-oCiWzbxh61hZLI?usp=sharing

#installing and importing pyomo
!pip install -q pyomo
from pyomo.environ import *

###installing and importing specific solvers (uncomment the one(s) you need)
###glpk
!apt-get install -y -qq glpk-utils
###cbc
#!apt-get install -y -qq coinor-cbc
###ipopt
#!wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
#!unzip -o -q ipopt-linux64
###bonmin
#!wget -N -q "https://ampl.com/dl/open/bonmin/bonmin-linux64.zip"
#!unzip -o -q bonmin-linux64
###couenne
#!wget -N -q "https://ampl.com/dl/open/couenne/couenne-linux64.zip"
#!unzip -o -q couenne-linux64
###geocode
#!wget -N -q "https://ampl.com/dl/open/gecode/gecode-linux64.zip"
#!unzip -o -q gecode-linux64

#Using the solvers:
#SolverFactory('glpk', executable='/usr/bin/glpsol')
#SolverFactory('cbc', executable='/usr/bin/cbc')
#SolverFactory('ipopt', executable='/content/ipopt')
#SolverFactory('bonmin', executable='/content/bonmin')
#SolverFactory('couenne', executable='/content/couenne')
#SolverFactory('gecode', executable='/content/gecode')

     |████████████████████████████████| 11.1 MB 16.1 MB/s 
     |████████████████████████████████| 49 kB 1.9 MB/s 
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 124015 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting u

In [ ]:
import pandas as pd

#SC Location Problem

In [ ]:
#reading in distribution centre data and previewing the same
dc_data = pd.read_excel('P4_Gundam.xlsx', sheet_name='DCdata')
dc_data

,location,capacity,fixedcost
0,Durham,20655,3831700
1,Fort Wayne,11279,1848400
2,St. Petersburg,25376,2190000
3,Laredo,40169,1361300
4,Buffalo,25167,1312000
5,Madison,12428,2168300
6,Lubbock,37810,1323800
7,Chandler,23909,1432500
8,Scottsdale,5493,1386600
9,Reno,25840,1577700


In [ ]:
#converting distribution centre data to a list of lists and also converting the columns into separate lists
distcenters = dc_data.loc[:, 'location':'fixedcost'].values.tolist()
location = dc_data["location"].tolist()
fixedcost = dc_data["fixedcost"].tolist()
capacity = dc_data["capacity"].tolist()

In [ ]:
#reading in city-wise demand data and previewing the same
citydata = pd.read_excel('P4_Gundam.xlsx', sheet_name='citydata') #Change to your file name and sheet name
citydata

,city,demand
0,New York,8622
1,Los Angeles,3999
2,Chicago,2716
3,Houston,3284
4,Phoenix,1626
...,...,...
71,Orlando,517
72,Irvine,338
73,Toledo,357
74,Jersey City,569


In [ ]:
#converting demand and city columns to a lists:
citydemand = citydata.loc[:,'city':'demand'].values.tolist()
demand = citydata["demand"].tolist()
cities = citydata["city"].tolist()

In [ ]:
#reading in the associated cost data and previewing the same
costdata = pd.read_excel('P4_Gundam.xlsx', sheet_name='distcost') #Change to your file name and sheet name
costdata

,Unnamed: 0,New York,Los Angeles,Chicago,Houston,Phoenix,Philadelphia,San Antonio,San Diego,Dallas,...,Anchorage,Greensboro,Plano,Newark,Lincoln,Orlando,Irvine,Toledo,Jersey City,Chula Vista
0,Durham,2693,1752,1820,2585,2861,3446,3814,3100,2098,...,2240,2169,2906,1592,3903,1351,3545,2851,1936,3710
1,Fort Wayne,2690,2581,1851,2175,1851,1895,2529,2525,1872,...,1042,2098,2770,1614,835,2652,694,294,1702,1341
2,St. Petersburg,2480,3128,2782,2213,2837,3252,2588,2735,1111,...,1521,2142,2392,500,1800,1270,1132,597,1730,3385
3,Laredo,5144,5693,3046,5593,4414,4492,5348,5757,4681,...,5324,5901,4510,3718,3165,5423,3204,3431,5895,4124
4,Buffalo,3436,4475,4340,5177,4286,4186,4459,4863,5337,...,5642,5111,5504,4217,4031,4532,5300,5761,4460,4240
5,Madison,3587,1973,2033,3632,935,1516,3159,3690,1811,...,3327,1242,2413,2880,2368,902,3515,2353,3686,1423
6,Lubbock,2404,794,2885,998,2791,1674,1414,133,2143,...,47,486,2743,1955,1302,500,1397,1173,1758,1622
7,Chandler,2805,2738,2750,3852,2442,3986,2293,2814,4996,...,3935,4184,4501,2995,3109,2585,5113,4117,2191,3099
8,Scottsdale,5486,5742,3576,4881,5403,3433,4993,5424,4196,...,3477,3824,5751,5908,5520,5247,3234,3079,3536,3174
9,Reno,1940,3552,3835,2819,3095,3953,2891,3931,3382,...,3509,4581,3216,3402,2129,2490,2408,3922,3355,2751


In [ ]:
#converting to a list:
distcost = costdata.loc[:, 'New York':'Chula Vista'].values.tolist()
distcost

[[2693,
  1752,
  1820,
  2585,
  2861,
  3446,
  3814,
  3100,
  2098,
  2113,
  3576,
  2312,
  3298,
  2423,
  3268,
  1754,
  1433,
  2504,
  2132,
  1368,
  3656,
  3182,
  2368,
  3817,
  1970,
  2430,
  1896,
  2058,
  2076,
  3375,
  2721,
  1831,
  3345,
  1245,
  1920,
  3707,
  1249,
  2722,
  3883,
  1670,
  1036,
  2156,
  2447,
  3725,
  2589,
  3766,
  2144,
  2100,
  980,
  2584,
  1690,
  3946,
  3390,
  2062,
  3830,
  3705,
  2660,
  2099,
  1897,
  1008,
  3041,
  1284,
  1903,
  1716,
  2956,
  3253,
  2240,
  2169,
  2906,
  1592,
  3903,
  1351,
  3545,
  2851,
  1936,
  3710],
 [2690,
  2581,
  1851,
  2175,
  1851,
  1895,
  2529,
  2525,
  1872,
  2910,
  2469,
  1590,
  2463,
  1272,
  1596,
  2596,
  1544,
  2682,
  1271,
  2412,
  1408,
  1154,
  649,
  1246,
  1815,
  2392,
  3167,
  2191,
  1977,
  1317,
  2151,
  280,
  1561,
  1749,
  1776,
  3064,
  1004,
  1710,
  1150,
  2697,
  998,
  2474,
  454,
  1641,
  1433,
  553,
  2226,
  2290,
  433,
  1461

In [ ]:
num_dc_loc = 23 #indexed by i
num_cities = 76 #indexed by j

bigM = 100000000

#define the optimization model
model = ConcreteModel()

#DVs
model.x = Var(range(num_dc_loc), domain = Binary) #loc SC in city i
model.y = Var(range(num_dc_loc), range(num_cities), domain = NonNegativeIntegers) #assign retail city j to SC i, model.y[j,i]

#Objective minimise fixed operating expenses plus distribution costs
total_distcost= sum(model.y[i,j]*distcost[i][j]for i in range(num_dc_loc) for j in range(num_cities))
total_fixedcost =  sum( model.x[i]*fixedcost[i] for i in range(num_dc_loc))
total_expenses = total_distcost + total_fixedcost
model.Objective = Objective(expr = total_expenses, sense = minimize)

#Constraint to check for the capacity
model.WithinCapacity = ConstraintList()
for i in range(num_dc_loc):
  model.WithinCapacity.add(sum(model.y[i,j] for j in range(num_cities)) <= model.x[i]*capacity[i] )

#Constraint to meet the demands
model.MeetDemand = ConstraintList()
for i in range(num_cities):
  model.MeetDemand.add(sum(model.y[j,i] for j in range(num_dc_loc)) >= demand[i] )

#Constraint to only assign DC to a city if a DC exists there
model.MaxAssign = ConstraintList()
for i in range(num_dc_loc):
  model.MaxAssign.add(sum(model.y[i,j] for j in range(num_dc_loc)) <= model.x[i]*bigM)

model.pprint()

7 Set Declarations
    MaxAssign_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   23 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23}
    MeetDemand_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   76 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76}
    WithinCapacity_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   23 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23}
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen 

In [ ]:
#solve the model
opt = SolverFactory('glpk')
opt.options['tmlim'] = 5 #specifies the time limit (in seconds)
opt.options['mipgap'] = .01 #specifies the optimality gap tolerance (.01 means can stop if <1% of optimal obj)
results = opt.solve(model, tee=True) #can set tee=True if you want to see the details.

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --tmlim 5 --mipgap 0.01 --write /tmp/tmpj579j29b.glpk.raw --wglp /tmp/tmpisynsuji.glpk.glp
 --cpxlp /tmp/tmp7ummep66.pyomo.lp
Reading problem data from '/tmp/tmp7ummep66.pyomo.lp'...
/tmp/tmp7ummep66.pyomo.lp:9741: warning: lower bound of variable 'x1' redefined
/tmp/tmp7ummep66.pyomo.lp:9741: warning: upper bound of variable 'x1' redefined
123 rows, 1772 columns, 4072 non-zeros
1771 integer variables, 23 of which are binary
9764 lines were read
Writing problem data to '/tmp/tmpisynsuji.glpk.glp'...
9613 lines were written
GLPK Integer Optimizer, v4.65
123 rows, 1772 columns, 4072 non-zeros
1771 integer variables, 23 of which are binary
Preprocessing...
23 constraint coefficient(s) were reduced
122 rows, 1771 columns, 4071 non-zeros
1771 integer variables, 23 of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.144e+06  ratio =  1.144e+06
GM: min|aij| =  4.566e-01  max|aij| =  2.190e+00  r

In [ ]:
#print the results
print("Total cost: ", model.Objective())
dc_location_solution = [model.x[i]() for i in range(num_dc_loc)]
city_demand_satisfaction_solution = [[model.y[i,j]() for i in range(num_dc_loc)] for j in range(num_cities)]


Total cost:  58006498.0


In [ ]:
pd.DataFrame(dc_location_solution, index = location, columns =["Assignment Value for the DC"])

,Assignment Value for the DC
Durham,0.0
Fort Wayne,0.0
St. Petersburg,0.0
Laredo,0.0
Buffalo,0.0
Madison,0.0
Lubbock,1.0
Chandler,0.0
Scottsdale,0.0
Reno,0.0


In [ ]:
#Store the value of distribution
distribution_data = [[model.y[j,i]()for i in range(num_cities)] for j in range(num_dc_loc)]

In [ ]:
# Store the above values in a dataframe and preview the results
city_dc = pd.DataFrame(distribution_data, index = location, columns =cities)
city_dc

,New York,Los Angeles,Chicago,Houston,Phoenix,Philadelphia,San Antonio,San Diego,Dallas,San Jose,...,Anchorage,Greensboro,Plano,Newark,Lincoln,Orlando,Irvine,Toledo,Jersey City,Chula Vista
Durham,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Fort Wayne,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
St. Petersburg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Laredo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Buffalo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Madison,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Lubbock,0.0,3999.0,0.0,3284.0,0.0,0.0,0.0,1419.0,0.0,1035.0,...,671.0,795.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chandler,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Scottsdale,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Reno,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#Total costs under the optimal solution
print("Total cost: ", model.Objective())

Total cost:  58006498.0


In [ ]:
#Exporting the results into a csv file.
city_dc.to_csv("P4_Gundam_Solutions.csv", header = True, index = False)